In [3]:
# Import package for Data Extraction, Loading & Transformation
import pandas as pd
import requests

# Import dotenv package for setting environment variables 
from dotenv import load_dotenv

# Import os package
import os

# Set environment variables from the .env in the local environment
load_dotenv()

# Retrieve API key and store as Python variable
api_key = os.getenv('API_KEY')
type(api_key)

str

In [4]:
# ETFs and Ticker Data for Sectoral Performance Analysis

ticker_name_dict = {'XIU':['iShares S&P/TSX 60 Index ETF','TSX_60'],
                    'XST':['iShares S&P/TSX Capped Consumer Staples Index ETF','Consumer Staples'],
                    'XHC':['iShares Global Healthcare Index ETF','Health'],
                    'XRE':['iShares S&P/TSX Capped REIT Index ETF','Real Estate'],
                    'XUT':['iShares S&P/TSX Capped Utilities Index ETF','Utilities'],
                    'CEW':['iShares Equal Weight Banc & Lifeco ETF','Financial Services'],
                    'TRVL':['Harvest Travel & Leisure Index ETF','Recreation & Leisure'],
                    'XGD':['iShares S&P/TSX Global Gold Index ETF','Gold ETF']}

ticker_df = pd.DataFrame.from_dict(ticker_name_dict,orient='index',
                                   columns=['ETF','Sector'])

ticker_df                                       

,ETF,Sector
XIU,iShares S&P/TSX 60 Index ETF,TSX_60
XST,iShares S&P/TSX Capped Consumer Staples Index ETF,Consumer Staples
XHC,iShares Global Healthcare Index ETF,Health
XRE,iShares S&P/TSX Capped REIT Index ETF,Real Estate
XUT,iShares S&P/TSX Capped Utilities Index ETF,Utilities
CEW,iShares Equal Weight Banc & Lifeco ETF,Financial Services
TRVL,Harvest Travel & Leisure Index ETF,Recreation & Leisure
XGD,iShares S&P/TSX Global Gold Index ETF,Gold ETF


In [5]:
# Creating New Directory to Save Data

os.makedirs('./Output', exist_ok = True)
ticker_df.to_csv('./Output/ticker.csv')

In [6]:
# Using Function to extract data using APIs and save as csvfiles

etf_list = ticker_df.index.to_list() # Creating a List of ETF Tickers


def extract_api_data(data_list):
    df_list = []
    for item in data_list:
        url = f'https://www.alphavantage.co/query?function=TIME_SERIES_MONTHLY&symbol={item}.TRT&outputsize=full&apikey={api_key}&datatype=csv'
        r = requests.get(url)
        df = pd.read_csv(url)
        df['timestamp'] = df['timestamp'].str[:4]
        item_df = df.rename(columns={'close': f'{item}','timestamp':'Year'})
        item_df = item_df.groupby('Year').mean()
        df_list.append(item_df)
    
    raw_df = pd.concat(df_list, axis=1)
    return(raw_df)

raw_etf_df = extract_api_data(etf_list)

combined_etf_df = raw_etf_df.drop(columns=['open','high','low','volume'],axis=1)

combined_etf_df.to_csv('./Output/etf_annual_data.csv')

display(combined_etf_df.head())
display(combined_etf_df.tail())


,XIU,XST,XHC,XRE,XUT,CEW,TRVL,XGD
Year,,,,,,,,
2005,57.660909,NaN,NaN,12.799045,NaN,NaN,NaN,51.521818
2006,68.781667,NaN,NaN,14.847333,NaN,NaN,NaN,75.980833
2007,79.343333,NaN,NaN,16.218783,NaN,NaN,NaN,74.636667
2008,54.714167,NaN,NaN,12.224200,NaN,7.15790,NaN,55.285833
2009,15.463333,NaN,NaN,9.569833,NaN,6.38115,NaN,20.005000


,XIU,XST,XHC,XRE,XUT,CEW,TRVL,XGD
Year,,,,,,,,
2020,24.128333,63.537500,53.970833,15.976717,27.167083,11.792233,NaN,20.140833
2021,29.879167,70.055000,63.690833,19.136183,29.904167,15.790767,21.826364,18.213333
2022,30.756667,80.149167,65.453333,18.015833,30.509167,15.635833,18.695833,17.397500
2023,30.694167,86.050833,65.910000,16.287500,26.570833,15.675000,21.104167,17.889167
2024,31.980000,91.130000,68.560000,16.200000,25.410000,16.410000,23.420000,16.210000


In [7]:
combined_etf_df

,XIU,XST,XHC,XRE,XUT,CEW,TRVL,XGD
Year,,,,,,,,
2005,57.660909,NaN,NaN,12.799045,NaN,NaN,NaN,51.521818
2006,68.781667,NaN,NaN,14.847333,NaN,NaN,NaN,75.980833
2007,79.343333,NaN,NaN,16.218783,NaN,NaN,NaN,74.636667
2008,54.714167,NaN,NaN,12.224200,NaN,7.157900,NaN,55.285833
2009,15.463333,NaN,NaN,9.569833,NaN,6.381150,NaN,20.005000
2010,17.665000,NaN,NaN,12.854233,NaN,7.440658,NaN,22.987500
2011,18.725833,20.377500,20.268750,15.053142,20.622500,7.652208,NaN,24.400833
2012,17.432500,22.569167,22.690000,16.859808,20.581667,7.222208,NaN,20.428333
2013,18.490833,28.423333,29.110000,16.316617,19.699167,8.835567,NaN,12.575833
